In [1]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
import numpy as np
import os
import time
from tqdm import tqdm
%matplotlib inline

In [29]:
df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/sales.csv',index_col=0)
df = df.drop_duplicates()
print('There are %.0f unique records in the database now' % df.shape[0])
print('Total revenue: %.02f' % df.amount_paid_usd.sum())

There are 100712 unique records in the database now
Total revenue: 758118.03


In [30]:
spotify_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/spotify_df.csv',index_col=0)

In [31]:
spotify_df.shape[0]

28050

In [32]:
matches = spotify_df[spotify_df.match == True]
matches.shape[0]

13737

In [33]:
artist_names = matches.bc_artist_name.values

df_matches = df[df.artist_name.isin(artist_names)]
urls_to_scrape = df_matches.url.values

for i,url in enumerate(urls_to_scrape):
    if url[0] == '/':
        urls_to_scrape[i] = 'https:' + url

Get urls already scraped:

In [34]:
urls_scraped = []
folder =  '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'
files = os.listdir(folder)

for i,file in enumerate(files):
    try:
        if not file[0] == '.':
            df_temp = pd.read_csv(os.path.join(folder,file))
            url = df_temp.album_url.values[0]
            if url[0] == '/':
                url = 'https:' + url
                print(url)
            
            urls_scraped.append(url)
    except:
        pass

print('%.0f albums have been scraped' % len(files))

3231 albums have been scraped


In [35]:
already = 0
notyet = 0
for url in urls_to_scrape:
    
    if url in urls_scraped:
        already+=1
    else:
        notyet+=1

In [36]:
already

11165

In [37]:
notyet

34788

In [38]:
urls_to_scrape[-1]

'https://thewarondrugs.bandcamp.com/album/lost-in-the-dream'

# LOOP THROUGH BANDCAMP SPIDER

In [39]:
cd '/Users/shayneufeld/github/pricecamp/scrapers/bandcamp_spider'

/Users/shayneufeld/GitHub/pricecamp/scrapers/bandcamp_spider


In [40]:
urls_to_scrape = np.asarray(urls_to_scrape)
urls_to_scrape = np.unique(urls_to_scrape)
urls_to_scrape = list(urls_to_scrape)

In [41]:
urls_to_scrape = ['https://thewarondrugs.bandcamp.com/album/lost-in-the-dream']
for i in urls_to_scrape:
    print(i)

https://thewarondrugs.bandcamp.com/album/lost-in-the-dream


In [42]:
urls_to_scrape = ['https://thewarondrugs.bandcamp.com/album/lost-in-the-dream']
urls_scraped = []

In [43]:
for url in urls_to_scrape:
    
    if (url not in urls_scraped):
        
        '''
        get input parameters
        '''
        if url[0] == 'h':
            album_url = url
        elif url[0] == '/':
            album_url = 'https:' + url
        else:
            print('URL BROKEN')
            #break
        artist_name = df[df.url==url].head(1).artist_name.values
        if not artist_name:
            artist_name = df[df.url==url[6:]].head(1).artist_name.values

        if type(artist_name) is not str:
            artist_name = artist_name[0]
            
        al_start = url.find('album')
            
        if al_start == -1:
            al_start = url.find('track')
        if al_start == -1:
            print(url)
        album_name = url[al_start+6:]
        
        #test to see if it has been scraped but has no entries:
        file_name = album_name + '-' + artist_name + '.csv'
        file_name = file_name.replace(' ','')
        
        if file_name in os.listdir(folder):
            print('%s already in folder' % file_name)
        elif 'merch' in album_url:
            print('%s is merch. Do not scrape' % file_name)
        else:
            print('%s new file!' % file_name)
            '''
            run bandcamp spider
            '''
            os.system("scrapy crawl bandcamp -a start_domain='%s' -a album_name='%s' -a artist_name='%s'" % (album_url,album_name,artist_name))
            #scrapy crawl bandcamp -a start_domain='https://andreabaroni.bandcamp.com/album/broken-space-ost' -a album_name='Broken Spaces' -a artist_name='Andrea Baroni'
            urls_scraped = np.append(urls_scraped,url)
            #np.savetxt('/Users/shayneufeld/Dropbox/insight/pricecamp/urls_scraped.csv',urls_scraped)
            print('%0f albums scraped' % len(os.listdir(folder)))
            time.sleep(1)

lost-in-the-dream-TheWarOnDrugs.csv new file!
3232.000000 albums scraped


In [64]:
list(urls_scraped)[:10]

['https://dreamcatalogue.bandcamp.com/album/--18',
 'https://telepathtelepath.bandcamp.com/album/--19',
 'http://lobstertheremin.com/track/--4',
 'https://grauenmetal.bandcamp.com/album/-',
 'https://zuriaakecn.bandcamp.com/album/-',
 'https://stefanvaldobrev.bandcamp.com/album/-',
 'https://m-a-d-e-s.bandcamp.com',
 'https://koantunes.bandcamp.com/track/06-koan-ananch-roeth-version',
 'https://filthycasualspod.bandcamp.com/album/10-games-to-play-before-you-die',
 'http://music.georgeclanton.com/album/100-electronica']

In [63]:
np.savetxt('/Users/shayneufeld/Dropbox/insight/pricecamp/urls_scraped',list(urls_scraped))

TypeError: Mismatch between array dtype ('<U157') and format specifier ('%.18e')

In [29]:
file_names = os.listdir(folder)

In [26]:
file_name

'/Users/shayneufeld/dropbox/insight/pricecamp/data/albums_fixed/that-positive-feeling-music-license-Alumo.csv'

In [32]:
file_name = album_name + '-' + artist_name + '.csv'
file_name

'that-positive-feeling-music-license-Alumo.csv'

In [33]:
file_name in file_names

True

In [ ]:
artist_name = df[df.url==url[6:]].head(1).artist_name.values
artist_name

In [31]:
file_names[100]

'adrift-in-sleepwakefulness-[Bucketheadland].csv'

In [ ]:
type(artist_name) is not str

In [ ]:
urls_scraped = []
folder =  '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'
files = os.listdir(folder)

for i,file in enumerate(files):
    try:
        if not file[0] == '.':
            df = pd.read_csv(os.path.join(folder,file))
            url = df.album_url.values[0]
            urls_scraped.append(url)
    except:
        pass

In [ ]:
urls_scraped

In [ ]:
d = {'url':urls_scraped}
urls_scraped_df = pd.DataFrame(data = d)
urls_scraped_df.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/urls_scraped.csv')